Name: Taseng Mancheykhun, PRN: 240840141021

In [ ]:
%%writefile shared_minmax.cu
#include <stdio.h>
#define N 1280
#define t 160

__global__ void findmaxmin(int *d_input, int *d_maxoutput, int *d_minoutput, int n)
{
    __shared__ int shareddata[t];
    int idx = blockIdx.x*blockDim.x + threadIdx.x;

    if (idx<n)
    {
      shareddata[threadIdx.x] = d_input[idx];
    }
    else
    {
      shareddata[threadIdx.x] = 0;
    }

    __syncthreads();

    if (threadIdx.x == 0)
    {
        int max=shareddata[0];
        int min=shareddata[0];

        for (int i=0;i<t;i++)
        {
            if (shareddata[i]>max)
            {
                max = shareddata[i];
            }
            else if (shareddata[i]<min)
            {
                min = shareddata[i];
            }
        }

        d_maxoutput[blockIdx.x] = max;
        d_minoutput[blockIdx.x] = min;
    }
}



int main()
{
  int h_input[N], h_maxoutput[N/t], h_minoutput[N/t];
  int *d_input, *d_maxoutput, *d_minoutput;
  int max,min;

  for (int i=0;i<N; i++)
  {
    h_input[i]=1+i;
  }

  cudaMalloc(&d_input, N*sizeof(int));


  cudaMalloc(&d_maxoutput, (N/t)*sizeof(int));


  cudaMalloc(&d_minoutput, (N/t)*sizeof(int));


  cudaMemcpy(d_input, h_input, N*sizeof(int), cudaMemcpyHostToDevice);

  findmaxmin<<<8,t>>>(d_input, d_maxoutput, d_minoutput, N);

  cudaMemcpy(h_maxoutput, d_maxoutput, (N/t) * sizeof(int), cudaMemcpyDeviceToHost);

  cudaMemcpy(h_minoutput, d_minoutput, (N/t) * sizeof(int), cudaMemcpyDeviceToHost);


  printf("Max of each block: \n");
  for (int i=0; i<N/t;i++)
  {
    printf("%d ",h_maxoutput[i]);
  }
  printf("\n\n");


  printf("Min of each block: \n");
  for (int i=0; i<N/t;i++)
  {
    printf("%d ",h_minoutput[i]);
  }
  printf("\n\n");


  max = h_maxoutput[0];
  min = h_minoutput[0];

  for (int i=0;i<N/t;i++)
  {
      if (h_maxoutput[i]>max)
      {
          max = h_maxoutput[i];
      }

      else if (h_minoutput[i]<min)
      {
          min = h_minoutput[i];
      }
  }

  printf("Maximum value is: %d\n", max);
  printf("Minimum value is: %d\n", min);


  cudaFree(d_input);
  cudaFree(d_maxoutput);
  cudaFree(d_minoutput);

  return 0;
}

Overwriting shared_minmax.cu


In [ ]:
!nvcc -o exe shared_minmax.cu
!./exe

Max of each block: 
160 320 480 640 800 960 1120 1280 

Min of each block: 
1 161 321 481 641 801 961 1121 

Maximum value is: 1280
Minimum value is: 1
